In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

## 前処理で使用する関数

In [2]:
def make_PCA_df(PCA_arr):#PCAで削減したものは二次元配列で帰ってくるので、それをデータフレームにして返す関数
    X=[0]*len(PCA_arr)
    Y=[0]*len(PCA_arr)
    index=0
    for arr in PCA_arr:
        X[index]=arr[0]
        Y[index]=arr[1]
        index+=1
    return pd.DataFrame({'X':X,'Y':Y})

def making_pred_df(df):#配当金、着の情報は切りぬかなくてもうまいことやってくれる。
    pred_race_df=df
    #pred_race_df=pred_race_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
    result_df=pred_race_df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})
    #配当金と結果の情報を横によけておく。(仮のデータフレームに格納する）=========================================================
    #クラスタリングに使わないカラムを取り除く
    drop_col_names=['result_com','money']
    train_drops_df=pd.DataFrame(columns=drop_col_names)
    train_drops_df['result_com']=result_df['result_com']
    train_drops_df['money']=result_df['money']
    result_df=result_df.drop(drop_col_names,axis=1)
    #==============================================================================================================================
    #ダミー変数化
    #月をダミー化
    empty_arr=[0]*len(result_df)
    for number in np.arange(1,13,1):
        result_df['month_{}'.format(int(number))]=empty_arr
    dummie_df=pd.get_dummies(result_df['month'])#月をダミー化
    for column, val in dummie_df.iteritems():
        result_df['month_{}'.format(int(column))]=val

        
    #季節のダミー化   
    empty_arr=[0]*len(result_df)
    seasons=['sp','su','au','wi']
    for season in seasons:
        result_df['{}'.format(number)]=empty_arr
    dummie_df=pd.get_dummies(result_df['season'])#季節をダミー化
    for column, val in dummie_df.iteritems():
        result_df['{}'.format(column)]=val


    #開催日数のダミー化
    empty_arr=[0]*len(result_df)
    ranges=[1,2,3,4,5,6,7]#年末がおかしくなるっぽいから、一応からむだけ作っておく　
    for number in ranges:
        result_df['range_{}'.format(int(number))]=empty_arr
    dummie_df=pd.get_dummies(result_df['range_date'])#開催日数をダミー化
    for column, val in dummie_df.iteritems():
        result_df['range_{}'.format(int(column))]=val
        
    #開催日数のうちの何日目かのダミー化
    empty_arr=[0]*len(result_df)
    num_dates=[1,2,3,4,5,6,7]
    for number in num_dates:
        result_df['num_date_{}'.format(int(number))]=empty_arr
    dummie_df=pd.get_dummies(result_df['num_date'])#開催日数をダミー化
    for column, val in dummie_df.iteritems():
        result_df['num_date_{}'.format(int(column))]=val
        
    result_df=result_df.drop(['month','season','range_date','num_date'],axis=1)#ダミー化し終わったカラムは削除する
    #レース番号のダミー化===============================================
    result_df_dummie=result_df
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1)    
    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        male_dummie_df=pd.get_dummies(result_df_dummie[col])#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    cols=list(result_df_dummie.columns)
    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
    #boat もmoterも番号は1~99とする
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
        for column, val in moter_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    #boat番号をダミー化
    for col in boat_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
        for column, val in boat_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
    result_df=result_df_dummie


    result_df['result_com']=train_drops_df['result_com']#正解ラベルを戻してあげる
    result_df['money']=train_drops_df['money']#配当金を戻してあげる
    model_df=result_df

    return model_df

## 前処理&学習、本番データ切り分け

In [1]:
result_df=pd.read_csv('../csv/train_asiya_date.csv',encoding='utf-8')
result_df=result_df.drop(["Unnamed: 0"],axis=1)

result_df=result_df.drop(['date','day'],axis=1)#現状使わないカラムは削除する

date_cols=['year','month','num_date','range_date','season']
date_df=pd.DataFrame(index=result_df.index)
for col in date_cols:
    date_df[col]=result_df[col]
result_df=result_df.drop('year',axis=1)#yearカラムはデータの切り抜きで使うが学習では使わないがので削除（関数内でdropされないから）
#result_df=result_df.drop(col,axis=1)
#money_col=result_df['money']
#result_df=result_df.drop('money',axis=1)#配当金情報の削除
#result_col=result_df['result_com']
#result_df=result_df.drop('result_com',axis=1)#着の組み合わせ
result_df=making_pred_df(result_df)#モデルに突っ込むだけの状態に加工（新人選手の情報、欠損地加工とダミー化）
#result_df.to_csv('train_df.csv')
df=result_df
### 学習データのベースを作成(validとtrainに分割)
#学習、テストデータ切り分け(2019,2020のデータを検証用データに、ほかは学習)
df['year']=date_df['year']
valid_df = df[(df['year']==2019) | ((df['year']==2020) )]#2019,2020のデータを検証用データに。
train_df =  df[(df['year']!=2019) & ((df['year']!=2020) )]#そのほかを学習データに
#学習データを切り分けたらyearはいらないから削除する
valid_df=valid_df.drop(['year'],axis=1)
train_df=train_df.drop(['year'],axis=1)

train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
valid_total_df=valid_df.copy()#あとで分析用に全部のまとめ先のDf
train_total_df=train_df.copy()#あとで分析用に全部のまとめ先のDf

NameError: name 'pd' is not defined

In [4]:
valid_df

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_92,racer_6_bo_93,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,result_com,money
15171,31.0,0.16,0.453,4,27.0,0.16,0.170,1,51.0,0.20,...,0,0,0,0,0,0,0,0,7,5780.0
15172,52.0,0.18,0.328,3,34.0,0.21,0.121,1,23.0,0.12,...,0,0,0,0,0,0,0,0,5,790.0
15173,38.0,0.16,0.525,4,57.0,0.22,0.078,2,34.0,0.16,...,0,0,0,0,0,0,0,0,6,390.0
15174,31.0,0.19,0.344,3,38.0,0.18,0.450,4,34.0,0.20,...,0,0,0,0,0,0,0,0,31,3060.0
15175,32.0,0.20,0.245,2,47.0,0.15,0.388,3,32.0,0.17,...,0,0,0,0,0,0,0,0,1,1360.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19337,51.0,0.19,0.156,2,42.0,0.13,0.508,4,44.0,0.17,...,0,0,0,0,0,0,0,0,83,54960.0
19338,44.0,0.17,0.426,3,33.0,0.15,0.308,2,49.0,0.16,...,0,0,0,0,0,0,0,0,14,650.0
19339,47.0,0.18,0.464,4,25.0,0.14,0.271,2,31.0,0.16,...,0,0,0,0,0,0,0,0,10,3930.0
19340,46.0,0.17,0.247,2,33.0,0.15,0.510,4,42.0,0.13,...,0,0,0,0,0,0,0,0,25,1430.0


In [5]:
train_df

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_92,racer_6_bo_93,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,result_com,money
0,35.0,0.14,0.541,4,40.0,0.19,0.287,2,30.0,0.19,...,0,0,0,0,0,0,0,0,6,1160.0
1,46.0,0.18,0.450,4,27.0,0.18,0.301,2,31.0,0.15,...,0,0,0,0,0,0,0,0,3,2960.0
2,43.0,0.16,0.480,3,35.0,0.21,0.255,2,42.0,0.17,...,0,0,0,0,0,0,0,0,44,21490.0
3,27.0,0.17,0.506,4,38.0,0.20,0.188,2,41.0,0.17,...,0,0,0,0,0,0,0,0,62,5100.0
4,32.0,0.21,0.439,3,25.0,0.17,0.536,4,45.0,0.16,...,0,0,0,0,0,0,0,0,25,1980.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15166,37.0,0.15,0.142,2,27.0,0.17,0.069,2,45.0,0.17,...,0,0,0,0,0,0,0,0,16,15230.0
15167,54.0,0.16,0.320,2,23.0,0.16,0.103,1,53.0,0.19,...,0,0,0,0,0,0,0,0,9,1350.0
15168,49.0,0.15,0.487,4,51.0,0.19,0.428,1,51.0,0.18,...,0,0,0,0,0,0,0,0,2,640.0
15169,47.0,0.14,0.425,4,49.0,0.17,0.287,2,33.0,0.16,...,0,0,0,0,0,0,0,0,21,5160.0


### target_comと、パラメータを渡してモデルを作成する関数

In [6]:
def making_model(result_com,depth,target_per):
    result_com
    print(result_com)
    
    #学習データのラベル変換==========================================================
    result_train_df=train_df.copy() 
    result_arr=[0]*len(result_train_df)
    i=0
    for result in result_train_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    result_train_df['result_com']=result_arr
    result_valid_df=valid_df.copy() 
    result_arr=[0]*len(result_valid_df)
    i=0
    for result in result_valid_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1

    result_valid_df['result_com']=result_arr

    result_train_df['money']=train_money
    result_valid_df['money']=valid_money
    #学習データラベル変換終わり============================================

    for_arr=np.arange(1,100,1)
    accuracy_arr=[0]*len(for_arr)
    target_per_arr=[0]*len(for_arr)
    pred_0=[0]*len(for_arr)
    gain_arr=[0]*len(for_arr)
    model_gain_arr=[0]*len(result_valid_df)
    valid_gain_arr=valid_money.values

    depths_arr=[4,5,6,7,8]

    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target_com']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
    RF = RF.fit(target_x_train,target_y_train)


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    display(model_score_s)
    
    return RF
            

## モデルの作成と検証、保存


In [16]:
#test_model=making_model(result_com,depth,target_per)
target=14
depth=6
per0=130
model=making_model(target,depth,per0)
model

#モデル保存
filename = '../pickle_data/date_model/model_com{}_dep{}_per{}_date.sav'.format(target,depth,per0)
pickle.dump(model, open(filename, 'wb'))


14


target_com       14.000000
depth             6.000000
target_per      130.000000
総収益           32460.000000
投資金額          25500.000000
出現数             108.000000
購買予測数           255.000000
利益率             127.294118
購買的中率             7.843137
的中数              20.000000
dtype: float64

In [8]:
#モデル保存
#filename = '../pickle_data/date_model/model_com{}_dep{}_per{}_date.sav'.format(target,depth,per0)
#pickle.dump(model, open(filename, 'wb'))

### pickleデータの動作確認

In [9]:
def pickle_model(result_com,depth,target_per):
    filename = '../pickle_data/date_model/model_com{}_dep{}_per{}_date.sav'.format(result_com,depth,target_per)
    result_com
    print(result_com)
    
    #学習データのラベル変換==========================================================
    result_train_df=train_df.copy() 
    result_arr=[0]*len(result_train_df)
    i=0
    for result in result_train_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    result_train_df['result_com']=result_arr
    result_valid_df=valid_df.copy() 
    result_arr=[0]*len(result_valid_df)
    i=0
    for result in result_valid_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1

    result_valid_df['result_com']=result_arr

    result_train_df['money']=train_money
    result_valid_df['money']=valid_money
    #学習データラベル変換終わり============================================

    for_arr=np.arange(1,100,1)
    accuracy_arr=[0]*len(for_arr)
    target_per_arr=[0]*len(for_arr)
    pred_0=[0]*len(for_arr)
    gain_arr=[0]*len(for_arr)
    model_gain_arr=[0]*len(result_valid_df)
    valid_gain_arr=valid_money.values

    depths_arr=[4,5,6,7,8]
    #for depth in depths_arr:
    #    for sum_target_per in tqdm(for_arr):

    #index=sum_target_per-1
    #target_per=50+sum_target_per
    #target_per_arr[index]=target_per

    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target_com']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = pickle.load(open(filename, 'rb'))


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #gain_arr[index]=sum(model_gain_arr)
    #accuracy_arr[index]=(count/num_1)*100
    #try:
    #    pred_0[index]=pred_valid_df['pred'].value_counts()[0]
    #except:
    #    pred_0[index]=0
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    display(model_score_s)
    return RF
            

In [10]:
target=7
depth=4
per0=130
model=pickle_model(target,depth,per0)
model

7


FileNotFoundError: [Errno 2] No such file or directory: '../pickle_data/date_model/model_com7_dep4_per130_date.sav'

In [ ]:
target=4
depth=7
per0=131
model=pickle_model(target,depth,per0)
model

In [ ]:
target=5
depth=8
per0=122
model=pickle_model(target,depth,per0)
model

In [ ]:
target=7
depth=7
per0=146
model=pickle_model(target,depth,per0)
model

In [ ]:
target=13
depth=6
per0=115
model=pickle_model(target,depth,per0)
model

In [ ]:
target=14
depth=4
per0=123
model=pickle_model(target,depth,per0)
model

In [ ]:
for i in target_x_valid.columns:
    print(i)